In [78]:
import numpy as np
import os,sys
import yaml
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import math
import cv2
import matplotlib.animation as animation

In [2]:
def wavefront_initialize(pixelsize_h=1e-6,pixelsize_v=1e-6,npixels_h=2048,npixels_v=2048,amplitude_value=0.0):
    #
    #create array at object (aperture) plane
    #
    amplitude = np.zeros((npixels_h,npixels_v))  # amplitude map
    amplitude += amplitude_value
    p_i_h = np.arange(npixels_h) * pixelsize_h
    p_x = (p_i_h - 0.5 * (p_i_h[-1] - p_i_h[0]) )
    p_i_v = np.arange(npixels_v) * pixelsize_v
    p_y = (p_i_v - 0.5 * (p_i_v[-1] - p_i_v[0]) )
    return p_x,p_y,amplitude

In [3]:
def wavefront_aperture(p_x,p_y,amplitude,diameter_h=40e-6,diameter_v=40e-6,diameter_r=40e-6,type=0):
    # aperture_type: 0=circular, 1=Square
    p_xx = p_x[:, np.newaxis]
    p_yy = p_y[np.newaxis, :]

    filter = np.zeros_like(amplitude)
    if type == 0:  # Circular aperture
        radius = (diameter_r/2)
        print("radius=%f um"%(1e6*radius))
        filter_illuminated_indices = np.where(p_xx**2 + p_yy**2 < radius**2)
        if filter_illuminated_indices[0].size ==0:
            print("Warning: wavefront_aperture(): Nothing goes trough the aperture")
        else:
            filter[filter_illuminated_indices] = 1.0
    elif type == 1:  # square
        radius_h = (diameter_h/2)
        radius_v = (diameter_v/2)
        print("radius_h=%f um,radius_v=%f um"%(1e6*radius_h,1e6*radius_v))
        filter_illuminated_indices = np.where( (np.abs(p_xx) < radius_h) & (np.abs(p_yy) < radius_v))
        if filter_illuminated_indices[0].size ==0:
            print("Warning: wavefront_aperture(): Nothing goes trough the aperture")
        else:
            filter[filter_illuminated_indices] = 1.0
    else:
        raise ValueError("Aperture type (shape) not valid")

    return p_x,p_y,amplitude*filter

In [4]:
with open('configure.yml','r') as conf_para:
    conf_para = yaml.load(conf_para,Loader=yaml.FullLoader)
#print(conf_para)
alpha_x = conf_para['Lens']['alpha_x']
alpha_y = conf_para['Lens']['alpha_y']
focus_x = conf_para['Lens']['focus_x']
focus_y = conf_para['Lens']['focus_y']

det_dist = conf_para['Exp_geom']['det_dist']
defocus = conf_para['Exp_geom']['defocus']
ap_x = conf_para['Lens']['ap_x']
ap_y = conf_para['Lens']['ap_y']

ss_size = conf_para['Detector']['ss_size']
fs_size = conf_para['Detector']['fs_size']

pixelsize_x = conf_para['Detector']['pixelsize_x']
pixelsize_y = conf_para['Detector']['pixelsize_y']
wl = conf_para['Source']['wl']
k = 2*np.pi/wl

In [5]:
p_x,p_y,pupil_input = wavefront_initialize(pixelsize_h=1e-7,pixelsize_v=1e-7,npixels_h=2048,npixels_v=2048,amplitude_value=1.0)
p_x,p_y,pupil_lens = wavefront_aperture(p_x,p_y,pupil_input,diameter_h=40e-6,diameter_v=40e-6,type=1)

radius_h=20.000000 um,radius_v=20.000000 um


In [179]:
Fpupil = np.fft.fftshift(np.fft.fft2(pupil_lens))
pxy = np.array(np.meshgrid(p_y,p_x))
alpha = -0.005
focus = 0.001*100
z = 0.001*100
wl = 7.29e-11
phi = np.pi/wl/z*(pxy[0]**2+pxy[1]**2)+alpha*1e9*(pxy[0]/focus)**3 +alpha*1e9*(pxy[1]/focus)**3
pupil_new = pupil_lens*np.exp(1.j*phi)
F = np.fft.fftshift(np.fft.fft2(pupil_new))
%matplotlib widget
x_min = np.min(p_x)
x_max = np.max(p_x)
y_min = np.min(p_y)
y_max = np.max(p_y)
plt.imshow(np.abs(F)**2,extent = [x_min,x_max,y_min,y_max])
#plt.imshow(pupil_lens,extent = [x_min,x_max,y_min,y_max])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [191]:
pxy = np.array(np.meshgrid(p_y,p_x))
alpha = -0.005
focus = 0.001*100
z = 0.001*600
wl = 7.29e-11
phi0 = -np.pi/wl/focus*(pxy[0]**2+pxy[1]**2)+alpha*1e9*(pxy[0]/focus)**3 +alpha*1e9*(pxy[1]/focus)**3
phi = np.pi/wl/z*(pxy[0]**2+pxy[1]**2)
pupil_new2 = pupil_lens*np.exp(1.j*(phi0+phi))
F2 = np.fft.fftshift(np.fft.fft2(pupil_new2))
%matplotlib widget
x_min = np.min(p_x)
x_max = np.max(p_x)
y_min = np.min(p_y)
y_max = np.max(p_y)
plt.imshow(np.abs(F2)**2,extent = [x_min,x_max,y_min,y_max])
#plt.imshow(pupil_lens,extent = [x_min,x_max,y_min,y_max])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [161]:
Fpupil = np.fft.fftshift(np.fft.fft2(pupil_lens))
pxy = np.array(np.meshgrid(p_y,p_x))
alpha = -0.005
zi = 0.002*100
za = 0.001*100
wl = 7.29e-11
phi = np.pi/wl/zi*(pxy[0]**2+pxy[1]**2)-np.pi/wl/za*(pxy[0]**2+pxy[1]**2)
pupil_new = pupil_lens*np.exp(1.j*phi)
F = np.fft.fftshift(np.fft.fft2(pupil_new))
%matplotlib widget
x_min = np.min(p_x)
x_max = np.max(p_x)
y_min = np.min(p_y)
y_max = np.max(p_y)
plt.imshow(np.abs(F),extent = [x_min,x_max,y_min,y_max])
#plt.imshow(pupil_lens,extent = [x_min,x_max,y_min,y_max])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [166]:
ims = []
fig = plt.figure()
for i in range(48):
    x_min = np.min(p_x)
    x_max = np.max(p_x)
    y_min = np.min(p_y)
    y_max = np.max(p_y)
    alpha = -0.005
    focus = 0.001*10
    z = 0.001*(1000-20*i)
    wl = 7.29e-11
    phi = np.pi/wl/focus*(pxy[0]**2+pxy[1]**2)+alpha*1e9*(pxy[0]/z)**3 +alpha*1e9*(pxy[1]/z)**3
    pupil_new = pupil_lens*np.exp(1.j*phi)
    F = np.fft.fftshift(np.fft.fft2(pupil_new))
    im = plt.imshow(np.abs(F)**2,extent = [x_min,x_max,y_min,y_max],animated=True)
    #plt.colorbar()
    #plt.savefig('zchange%f_%s.png'%(z,i))
    #print(z)
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True,repeat_delay=1000)
ani.save('try2.mp4')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [94]:
ims = []
fig = plt.figure()
for i in range(48):
    x_min = np.min(p_x)
    x_max = np.max(p_x)
    y_min = np.min(p_y)
    y_max = np.max(p_y)
    alpha = -0.005
    focus = 0.001
    z = 0.001*(1000-20*i)
    wl = 7.29e-11
    phi = np.pi/wl/focus*(pxy[0]**2+pxy[1]**2)-np.pi/wl/z*(pxy[0]**2+pxy[1]**2)
    pupil_new = pupil_lens*np.exp(1.j*phi)
    F = np.fft.fftshift(np.fft.fft2(pupil_new))
    im = plt.imshow(np.abs(F)**2,extent = [x_min,x_max,y_min,y_max],animated=True)
    #plt.colorbar()
    #plt.savefig('zchange%f_%s.png'%(z,i))
    #print(z)
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True,repeat_delay=1000)
ani.save('try2_spherical.mp4')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [164]:
ims = []
fig = plt.figure()
for i in range(50):
    pxy = np.array(np.meshgrid(p_y,p_x))
    alpha = -0.005
    zi = 0.001*100
    za = 0.002*(10+i*2)
    wl = 7.29e-11
    phi = np.pi/wl/zi*(pxy[0]**2+pxy[1]**2)-np.pi/wl/za*(pxy[0]**2+pxy[1]**2)
    pupil_new = pupil_lens*np.exp(1.j*phi)
    F = np.fft.fftshift(np.fft.fft2(pupil_new))
    im = plt.imshow(np.abs(F)**2,extent = [x_min,x_max,y_min,y_max],animated=True)
    #plt.colorbar()
    #plt.savefig('zchange%f_%s.png'%(z,i))
    #print(z)
    ims.append([im])
ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True,repeat_delay=1000)
ani.save('try3_spherical.mp4')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [198]:
ims = []
fig = plt.figure()
for i in range(50):
    pxy = np.array(np.meshgrid(p_y,p_x))
    alpha = -0.005
    focus = 0.001*100
    z = 0.001*(20*(i+1))
    wl = 7.29e-11
    phi0 = -np.pi/wl/focus*(pxy[0]**2+pxy[1]**2)+alpha*1e9*(pxy[0]/focus)**3 +alpha*1e9*(pxy[1]/focus)**3
    phi = np.pi/wl/z*(pxy[0]**2+pxy[1]**2)
    pupil_new2 = pupil_lens*np.exp(1.j*(phi0+phi))
    F2 = np.fft.fftshift(np.fft.fft2(pupil_new2))
    im = plt.imshow(np.abs(F2)**2,extent = [x_min,x_max,y_min,y_max],animated=True)
    ims.append([im])
ani = animation.ArtistAnimation(fig, ims, interval=100, blit=True,repeat_delay=1000)
ani.save('try4_aberration.mp4')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …